In [4]:
!pip install urlexpander


In [5]:
import urllib3, socket
from urllib3.connection import HTTPConnection

HTTPConnection.default_socket_options = ( 
            HTTPConnection.default_socket_options + [
            (socket.SOL_SOCKET, socket.SO_SNDBUF, 1000000000), #1MB in byte
            (socket.SOL_SOCKET, socket.SO_RCVBUF, 1000000000)
        ])

In [18]:
import pandas as pd
import urlexpander
import requests

df = pd.read_csv('/content/urlsharing.csv')
df.head(3)

,url,user.screen_name,NumberOfUsersShareSameUrl,uniqueUserNameShareSameUrl
0,https://support.twitter.com/articles/20169199,"['JasonIsGone_', 'AadiK36769152', 'JasonIsGone...",8,"['Elfunadoreturns', 'LecolleurP', 'JasonIsGone..."
1,https://t.co/008uEAJnId,"['CherylD59995761', 'CherylD59995761']",2,['CherylD59995761']
2,https://t.co/00D3q4f9Yx,['Om39472295'],1,['Om39472295']


In [7]:
## divide dataframe into 7 dataframe
# df1 = df[:3000]
# df2 = df[3000:6000]


In [8]:
df['unShortenUrl'] = 0
df['frequency'] = 0

In [9]:
df['url'][0]

'https://support.twitter.com/articles/20169199'

In [10]:
df['url'][21]

'https://t.co/048RIjLkOh'

In [11]:
df.columns

Index(['url', 'user.screen_name', 'NumberOfUsersShareSameUrl',
       'uniqueUserNameShareSameUrl', 'unShortenUrl', 'frequency'],
      dtype='object')

In [12]:
print("shape of this data is : " , df.shape)

shape of this data is :  (19491, 6)


In [13]:
urls = list(df['url'])
print(len(urls[0]))

45


In [14]:
def unshorten_one(url):
    try:
        short = urlexpander.expand(url)
    except requests.exceptions.MissingSchema:
        short = urlexpander.expand("http://" + url)
    except requests.ConnectionError:
        short = "connection error"
    except requests.TooManyRedirects:
        short = "__CLIENT_ERROR__"
        
    if "__CLIENT_ERROR__" in short or "__CONNECTIONPOOL_ERROR__"  in short:
        try:
            r = requests.get(url, allow_redirects=False)
            try:
                short = r.headers["location"]
            except KeyError:
                short = "Page doesn't exist!"
            except requests.TooManyRedirects:
                short = urlexpander.expand(url)
        except requests.exceptions.MissingSchema:
            r = requests.get("http://" + url, allow_redirects=False)
            try:
                short = r.headers["location"]
            except KeyError:
                short = "Page doesn't exist!"
        except requests.ConnectionError:
            short = "connection error"
            

    return short

In [25]:
print(unshorten_one('https://t.co/f0sbDFQETQ'))

https://www.ibtimes.com/amber-heard-allegedly-lied-about-johnny-depp-blowout-fight-his-lawyer-claims-3182598


In [15]:
not_ex1 = []
j = 0
for i in urls:
    print(j)
    not_ex1.append(unshorten_one(i))
    j = j +1

Streaming output truncated to the last 5000 lines.
7792
7793
7794
7795
7796
7797
7798
7799
7800
7801
7802
7803
7804
7805
7806
7807
7808
7809
7810
7811
7812
7813
7814
7815
7816
7817
7818
7819
7820
7821
7822
7823
7824
7825
7826
7827
7828
7829
7830
7831
7832
7833
7834
7835
7836
7837
7838
7839
7840
7841
7842
7843
7844
7845
7846
7847
7848
7849
7850
7851
7852
7853
7854
7855
7856
7857
7858
7859
7860
7861
7862
7863
7864
7865
7866
7867
7868
7869
7870
7871
7872
7873
7874
7875
7876
7877
7878
7879
7880
7881
7882
7883
7884
7885
7886
7887
7888
7889
7890
7891
7892
7893
7894
7895
7896
7897
7898
7899
7900
7901
7902
7903
7904
7905
7906
7907
7908
7909
7910
7911
7912
7913
7914
7915
7916
7917
7918
7919
7920
7921
7922
7923
7924
7925
7926
7927
7928
7929
7930
7931
7932
7933
7934
7935
7936
7937
7938
7939
7940
7941
7942
7943
7944
7945
7946
7947
7948
7949
7950
7951
7952
7953
7954
7955
7956
7957
7958
7959
7960
7961
7962
7963
7964
7965
7966
7967
7968
7969
7970
7971
7972
7973
7974
7975
7976
7977
7978
7979
7980
7981

UnicodeDecodeError: ignored

In [17]:
print(len(not_ex1))

12791


In [19]:
df = df[:12791]
df.shape

(12791, 4)

In [20]:
df['unShortenUrl'] = not_ex1

In [21]:
df.to_csv('dff-output.csv' , index = False)

In [22]:
df.tail(5)

,url,user.screen_name,NumberOfUsersShareSameUrl,uniqueUserNameShareSameUrl,unShortenUrl
12786,https://t.co/eziY2SO21T,['Abogado_PH'],1,['Abogado_PH'],https://abogado.com.ph/amber-heard-vindicated-...
12787,https://t.co/f0IItQlCxl,['Teda_Singh'],1,['Teda_Singh'],https://twitter.com/Shashantarakian/status/137...
12788,https://t.co/f0c6rKyNF2,['MarciaRodmell'],1,['MarciaRodmell'],https://www.youtube.com/watch?v=MA_trAwSP00&fe...
12789,https://t.co/f0p9qd5NMo,['RegardNews'],1,['RegardNews'],https://regardnews.com/amber-heard-vs-johnny-d...
12790,https://t.co/f0sbDFQETQ,['anomaly81303798'],1,['anomaly81303798'],https://www.ibtimes.com/amber-heard-allegedly-...


In [23]:
!cp -r /content/dff-output.csv /content/drive/MyDrive/urlshorten/output/

In [26]:
df = pd.read_csv('/content/urlSharing-unshorten.csv')
df.head(4)

,url,user.screen_name,NumberOfUsersShareSameUrl,uniqueUserNameShareSameUrl,unShortenUrl
0,https://support.twitter.com/articles/20169199,"['JasonIsGone_', 'AadiK36769152', 'JasonIsGone...",8,"['Elfunadoreturns', 'LecolleurP', 'JasonIsGone...",https://help.twitter.com/en/rules-and-policies...
1,https://t.co/008uEAJnId,"['CherylD59995761', 'CherylD59995761']",2,['CherylD59995761'],https://www.youtube.com/watch?v=Ut40yEFetK8&fe...
2,https://t.co/00D3q4f9Yx,['Om39472295'],1,['Om39472295'],https://www.youtube.com/watch?v=sSu_bvw9qKo&fe...
3,https://t.co/00Vfekd0aP,['TriereGorum'],1,['TriereGorum'],https://www.change.org/p/dc-entertainment-remo...


In [36]:
df.columns

Index(['url', 'user.screen_name', 'NumberOfUsersShareSameUrl',
       'uniqueUserNameShareSameUrl', 'unShortenUrl', 'frequency'],
      dtype='object')

In [37]:
len(list(df['user.screen_name'][0]))

131

In [27]:
df.shape

(12791, 5)

In [31]:
df['frequency'] = 0
for i in range(len(df)):
    df['frequency'][i] = len(df['uniqueUserNameShareSameUrl'][i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [32]:
df.head(5)

,url,user.screen_name,NumberOfUsersShareSameUrl,uniqueUserNameShareSameUrl,unShortenUrl,frequency
0,https://support.twitter.com/articles/20169199,"['JasonIsGone_', 'AadiK36769152', 'JasonIsGone...",8,"['Elfunadoreturns', 'LecolleurP', 'JasonIsGone...",https://help.twitter.com/en/rules-and-policies...,83
1,https://t.co/008uEAJnId,"['CherylD59995761', 'CherylD59995761']",2,['CherylD59995761'],https://www.youtube.com/watch?v=Ut40yEFetK8&fe...,19
2,https://t.co/00D3q4f9Yx,['Om39472295'],1,['Om39472295'],https://www.youtube.com/watch?v=sSu_bvw9qKo&fe...,14
3,https://t.co/00Vfekd0aP,['TriereGorum'],1,['TriereGorum'],https://www.change.org/p/dc-entertainment-remo...,15
4,https://t.co/00jDf9wcWW,['donaldherring11'],1,['donaldherring11'],https://www.tmz.com/2021/03/18/johnny-depp-cla...,19


In [38]:
df.drop(['frequency'] , axis = 1 , inplace = True)

In [39]:
df.to_csv("urlSharingAndShorten.csv" , index = False)